In [1]:
import pandas as pd
%load_ext sql

Connect to the database, load up HUD's section 8 income limits and all the counties

In [2]:
%%sql postgresql://matt@localhost/geocoder
select 1;

1 rows affected.


?column?
1


In [3]:
hud_data = pd.read_excel('../arp_pipeline/data/hud/Section8-FY21.xlsx', dtype='object')

In [4]:
county_query = %sql select county.cntyidfp, county.statefp, county.countyfp, county.name, stusps from tiger.county county, tiger_data.state_all state where county.statefp = state.statefp; 

 * postgresql://matt@localhost/geocoder
3233 rows affected.


In [5]:
tiger_data = county_query.DataFrame()

In [6]:
tiger_data['fips2010'] = tiger_data.apply(lambda row: f"{row.cntyidfp}99999", axis=1)

In [7]:
hud_data.shape

(4766, 36)

In [8]:
hud_data[hud_data.State_Alpha=='DC'].head()

,State_Alpha,fips2010,State,County,cbsasub,Metro_Area_Name,County_Name,median2021,l50_1,l50_2,...,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,MSA,county_town_name,state_name,metro
480,DC,1100199999,11,1,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",District of Columbia,129000,45150,51600,...,74100,82300,88900,95500,102100,108650,8840,District of Columbia,District of Columbia,1


In [9]:
hud_data[hud_data.State_Alpha=='VA'].sort_values('Metro_Area_Name')

,State_Alpha,fips2010,State,County,cbsasub,Metro_Area_Name,County_Name,median2021,l50_1,l50_2,...,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,MSA,county_town_name,state_name,metro
4360,VA,5100199999,51,1,NCNTY51001N51001,"Accomack County, VA",Accomack County,57500,20900,23900,...,43000,47750,51600,55400,59250,63050,9999,Accomack County,Virginia,0
4362,VA,5100599999,51,5,NCNTY51005N51005,Alleghany County-Clifton Forge city-Covington ...,Alleghany County,60700,21250,24300,...,43700,48550,52450,56350,60250,64100,9999,Alleghany County,Virginia,0
4461,VA,5158099999,51,580,NCNTY51005N51005,Alleghany County-Clifton Forge city-Covington ...,Covington city,60700,21250,24300,...,43700,48550,52450,56350,60250,64100,9999,Covington city,Virginia,0
4368,VA,5101799999,51,17,NCNTY51017N51017,"Bath County, VA",Bath County,61900,21850,25000,...,44950,49900,53900,57900,61900,65900,9999,Bath County,Virginia,0
4483,VA,5175099999,51,750,METRO13980M13980,"Blacksburg-Christiansburg-Radford, VA HUD Metr...",Radford city,79700,27900,31900,...,57400,63750,68850,73950,79050,84150,9999,Radford city,Virginia,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4492,VA,5184099999,51,840,METRO49020M49020,"Winchester, VA-WV MSA",Winchester city,81400,28500,32600,...,58600,65100,70350,75550,80750,85950,9999,Winchester city,Virginia,1
4393,VA,5106999999,51,69,METRO49020M49020,"Winchester, VA-WV MSA",Frederick County,81400,28500,32600,...,58600,65100,70350,75550,80750,85950,9999,Frederick County,Virginia,1
4452,VA,5119599999,51,195,NCNTY51195N51195,"Wise County-Norton city, VA HUD Nonmetro FMR Area",Wise County,49000,20900,23900,...,43000,47750,51600,55400,59250,63050,9999,Wise County,Virginia,0
4479,VA,5172099999,51,720,NCNTY51195N51195,"Wise County-Norton city, VA HUD Nonmetro FMR Area",Norton city,49000,20900,23900,...,43000,47750,51600,55400,59250,63050,9999,Norton city,Virginia,0


# Are the FIPS2010 codes just County Codes in the Tiger Data?
Okay, so HUD has 4766 rows, but there are only 3234 counties. First question: How many of HUDs "fips2010" IDs are just counties, vs how many aren't?

In [10]:
hud_and_tiger = pd.merge(left=hud_data, right=tiger_data, on='fips2010', how='left')

In [25]:
hud_and_tiger[pd.isnull(hud_and_tiger.statefp)].value_counts('State_Alpha')

State_Alpha
ME    533
MA    351
NH    259
VT    255
CT    169
RI     39
AS      1
MO      1
MP      1
dtype: int64

In [12]:
hud_and_tiger.shape

(4766, 41)

## Answer: _Mostly!_
So, for all but the above states, the FIPs 2010 codes are really just the county ID's in TIGER. 

The ones that have a single missing county give me pause, but let's look randomly at one in ME and see if it's a `County Subdivision`

# Are the remaining FIPS2010 codes Tiger County Subdivisions?

In [13]:
hud_data[hud_data.State_Alpha=='ME'].head()

,State_Alpha,fips2010,State,County,cbsasub,Metro_Area_Name,County_Name,median2021,l50_1,l50_2,...,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,MSA,county_town_name,state_name,metro
1338,ME,2300102060,23,1,METRO30340M30340,"Lewiston-Auburn, ME MSA",Androscoggin County,71200,24950,28500,...,51300,56950,61550,66100,70650,75200,4240,Auburn city,Maine,1
1339,ME,2300119105,23,1,METRO30340M30340,"Lewiston-Auburn, ME MSA",Androscoggin County,71200,24950,28500,...,51300,56950,61550,66100,70650,75200,9999,Durham town,Maine,1
1340,ME,2300129255,23,1,METRO30340M30340,"Lewiston-Auburn, ME MSA",Androscoggin County,71200,24950,28500,...,51300,56950,61550,66100,70650,75200,4240,Greene town,Maine,1
1341,ME,2300138565,23,1,METRO30340M30340,"Lewiston-Auburn, ME MSA",Androscoggin County,71200,24950,28500,...,51300,56950,61550,66100,70650,75200,9999,Leeds town,Maine,1
1342,ME,2300138740,23,1,METRO30340M30340,"Lewiston-Auburn, ME MSA",Androscoggin County,71200,24950,28500,...,51300,56950,61550,66100,70650,75200,4240,Lewiston city,Maine,1


In [14]:
%sql select * from tiger.cousub where cosbidfp = '2300119105';

 * postgresql://matt@localhost/geocoder
1 rows affected.


[(12983, '23', '001', '19105', '00582448', '2300119105', 'Durham', 'Durham town', '43', 'T1', 'G4040', '775', '76750', None, 'A', Decimal('99145269'), Decimal('1976257'), '+43.9560209', '-070.1324175', '0106000020AD100000010000000103000000010000006201000093FDF334608E51C0F6B52E3542FD454050A73CBA118E51C090A0F831E6FC454083C2A04CA38D51C0FE5F75E448FD45401 ... (11082 characters truncated) ... D51C05643E21E4BFB454098C28366D78D51C09A594B0169FB454099B7EA3A548E51C0B01D8CD827FC454078B306EFAB8E51C0001B1021AEFC454093FDF334608E51C0F6B52E3542FD4540')]

Promising, let's see how many line up for real

In [15]:
cousub_query = %sql select cousub.cosbidfp, cousub.gid, cousub.namelsad from tiger.cousub cousub, tiger.state_lookup state where state.statefp=cousub.statefp and state.abbrev in ('ME', 'MA', 'NH', 'VT', 'CT', 'RI')

 * postgresql://matt@localhost/geocoder
1618 rows affected.


In [16]:
cousubs = cousub_query.DataFrame()

In [17]:
cousubs.head()

,cosbidfp,gid,namelsad
0,2502328285,2295,Hanover town
1,2502338855,2296,Marshfield town
2,2502350145,2297,Norwell town
3,2502372985,2298,Wareham town
4,2500300975,2299,Alford town


In [26]:
hud_strange_subset = hud_data.loc[hud_data.State_Alpha.isin(hud_and_tiger[pd.isnull(hud_and_tiger.statefp)].value_counts('State_Alpha').index)]

In [27]:
hud_cousubs = pd.merge(left=hud_strange_subset, right=cousubs, left_on='fips2010', right_on='cosbidfp', how='left')

In [28]:
hud_cousubs[pd.isnull(hud_cousubs.gid)].value_counts('State_Alpha')

State_Alpha
MO    116
AS      1
MP      1
dtype: int64

## Answer: _Mostly!_

So for we have 10 codes that aren't county subdivisions in the north-east. The way I did this, those might just be counties, so let's look

In [21]:
hud_cousubs[pd.isnull(hud_cousubs.gid)]

,State_Alpha,fips2010,State,County,cbsasub,Metro_Area_Name,County_Name,median2021,l50_1,l50_2,...,l80_6,l80_7,l80_8,MSA,county_town_name,state_name,metro,cosbidfp,gid,namelsad


In [22]:
len('2300302760')

10

Whelp, that's too bad --- those are still pretty confusing since they're not counties (they don't end in 99999) and they're not county subdivisions either. They're all 10-digits, which would suggest they _should_ be county subdivisions, which makes me wonder if they're simply older or newer than the 2010 data I loaded.

But, doing a little more digging, I found the (official list of Maine County Subdivisions on census.gov)[https://www2.census.gov/geo/docs/reference/codes/files/st23_me_cousub.txt] and sure enough,  2300302760, 2300311020, and 2300356205 are all in there. It's unclear to me why those _aren't_ in my tiger data tho. Maybe they're in some older dataset?

In [23]:
(4766-14)/4766

0.9970625262274444

# So, where do we go from here?

My intermediate product (which will be used to is going to need 

* Addresess
* fips2010
* census tract
* AMI
* Tract median income

So, if I wanted an intermediate table to make that easier, I could do

* fips2010 (which is the PK of HUD's data, effectively)
* fkey to hud data (assuming I don't use the fips2010 as an fkey, which I probably won't?)
* state fips
* county fips
* fkey to tiger.county
* census tract if the fips2010 links to it
* fkey to tiger.tract
* area geometry (regardless of county or tract)

Then I can query addresses in there and pretty quickly get to the other data I want